In [ ]:
from init_notebook import *

from src.models.clip import ClipSingleton

In [ ]:
from experiments.datasets import PixelartDataset
ds = PixelartDataset()

In [ ]:

embeddings = []
for batch, label in tqdm(DataLoader(ds, batch_size=32)):
    batch = VF.resize(batch, (224, 224), VF.InterpolationMode.BILINEAR, antialias=True)
    embedding_batch = ClipSingleton.encode_image(batch)
    embeddings.append(embedding_batch.cpu())
    #break

embeddings = torch.concat(embeddings)
print(embeddings.shape)

In [ ]:
torch.save(embeddings, "../../pixelart-dataset/datasets/v1/clip-vit-b32-embeddings.pt")

# use

In [ ]:
embeddings = torch.load("../../pixelart-dataset/datasets/v2/clip-vit-b32-embeddings.pt")
embeddings = normalize_embedding(embeddings).float()

In [ ]:
prompts = [
    "brick wall", "stone", "water", "grass", "sky", "sword", "monster", "dragon", "women", "fire",
]
prompt_embeddings = ClipSingleton.encode_text(prompts, device="cpu", normalize=True)
print(prompt_embeddings.shape)

sim = prompt_embeddings @ embeddings.T
print(sim.shape)

indices = sim.argsort(dim=1, descending=True)
grid = [
    ds[int(i)][0] for i in indices[:, :16].T.flatten(0)
]
VF.to_pil_image(make_grid(grid, nrow=indices.shape[0]))